# Exploring Survey Definitions

The purpose of this notebook is to demonstrate how to load the parameters of the survey definitions and explore how to use them.

In [1]:
from os import path, getcwd
from sys import path as pythonpath
pythonpath.append(path.join(getcwd(), '..'))
import config_utils
import regions

Alternative designs for the Roman Galactic Plane Survey are described in the file config/rgps_survey_definitions.json.  This can be loaded into a Python dictionary using the config_utils provided. 

Each survey design is described as a Python dictionary, with entries for the different optical elements to be used to observe different regions.  Multiple regions can be specified for each optical element.

In [2]:
survey_def_file = path.join(getcwd(), '..', 'config', 'rgps_survey_definitions.json')
survey_def = config_utils.read_config(survey_def_file)
survey_def

{'Option 1a (Wide field=65%)': {'F213': [{'l': [10.0, 42.4], 'b': [-2.0, 2.0]},
   {'l': [-10.0, 10.0], 'b': [-10.0, 10.0]},
   {'l': [317.6, 350.0], 'b': [-2.0, 2.0]}],
  'F129': [{'l': [10.0, 42.4], 'b': [-2.0, 2.0]},
   {'l': [-10.0, 10.0], 'b': [-10.0, 10.0]},
   {'l': [317.6, 350.0], 'b': [-2.0, 2.0]}],
  'F158': [{'l': [10.0, 27.3], 'b': [-2.0, 2.0]},
   {'l': [-10.0, 10.0], 'b': [-10.0, 10.0]},
   {'l': [332.7, 350.0], 'b': [-2.0, 2.0]}],
  'F106': [{'l': [10.0, 27.3], 'b': [-2.0, 2.0]},
   {'l': [-10.0, 10.0], 'b': [-10.0, 10.0]},
   {'l': [332.7, 350.0], 'b': [-2.0, 2.0]}],
  'comment': 'Includes northern plane, bulge and southern plane'},
 'Option 1b (Wide field=85%)': {'F213': [{'l': [10.0, 70.9], 'b': [-2.0, 2.0]},
   {'l': [-10.0, 10.0], 'b': [-10.0, 10.0]},
   {'l': [289.1, 350.0], 'b': [-2.0, 2.0]}],
  'F129': [{'l': [10.0, 70.9], 'b': [-2.0, 2.0]},
   {'l': [-10.0, 10.0], 'b': [-10.0, 10.0]},
   {'l': [289.1, 350.0], 'b': [-2.0, 2.0]}],
  'F158': [{'l': [10.0, 42.6], 'b

For the purposes of evaluating survey maps, metrics etc, this package uses CelestialRegion objects, which provide a range of functionality.  Rather than have to re-create these objects from the original survey design every time, the conversion to CelestialRegions has already been made and stored in the file config/rgps_survey_definition_regions.json.  
From here, they can be loaded directly as follows.

In [4]:
# First we need to load the overall parameters of the simulation
sim_config = config_utils.read_config(path.join(getcwd(), '..', 'config', 'sim_config.json'))

survey_regions_file = path.join(getcwd(), '..', 'config', 'rgps_survey_definition_regions.json')
survey_regions = regions.load_regions_from_file(sim_config, survey_regions_file)
survey_regions

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



AttributeError: 'list' object has no attribute 'items'